<h2>You can see your computer specifications here </h2>

In [ ]:
!nvidia-smi

In [ ]:
from ultralytics import YOLO
import carla, cv2, time, numpy as np, atexit
from pathlib import Path
import matplotlib.pyplot as plt
from simulation.python_3_8_20_scripts.shared_memory_utils import CarlaWrapper

# Choose model
model_type = "yolo11n.pt"
model = YOLO(model_type) # Load a pretrained YOLO.. model

# === 2) Geef hier je afbeeldingsmap op ===
img_dir = Path("/tmp/pycharm_project_481/Data/Paar")  # <-- pas dit aan

# === 3) Verzamel alle afbeeldingspaden ===
img_paths = sorted(img_dir.glob("*.jpg")) + sorted(img_dir.glob("*.png"))
print(f"Gevonden {len(img_paths)} afbeeldingen.")



In [ ]:
# === 4) YOLO run op alle afbeeldingen (niet opslaan, niet showen) ===
results = model(
    source=[str(p) for p in img_paths],
    conf=0.4,
    save=False,
    show=False
)
print(type(results[0]))

# === 5) Toon alle detectieresultaten ===
for i, result in enumerate(results):
    res_img = result.plot()  # Tekent bounding boxes
    plt.figure(figsize=(8, 6))
    plt.imshow(res_img)
    plt.axis("off")
    plt.title(f"Detectieresultaat afbeelding {i+1}: {img_paths[i].name}")
    plt.show()
    print(type(res_img))

<h3>Section below is for live feed</h3>

In [ ]:
# --------------------------------------------------------------
# Helper: convert CARLA image (BGRA) → RGB NumPy array
# --------------------------------------------------------------
def carla_image_to_rgb(image):
    # Convert raw bytes into a NumPy array
    arr = np.frombuffer(image.raw_data, dtype=np.uint8)
    # Reshape to height × width × 4 (BGRA channels)
    arr = arr.reshape((image.height, image.width, 4))
    # Keep only the first 3 channels and flip BGR→RGB
    return arr[:, :, :3][:, :, ::-1]

# --------------------------------------------------------------
# Connect to the CARLA simulator
# --------------------------------------------------------------
client = carla.Client("localhost", 2000)   # connect to server
client.set_timeout(10.0)
world = client.get_world()                 # get current world

# --------------------------------------------------------------
# Enable synchronous mode for stable frame rate
# --------------------------------------------------------------
settings = world.get_settings()
if not settings.synchronous_mode:
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 1 / 20.0  # 20 FPS simulation step
    world.apply_settings(settings)

# --------------------------------------------------------------
# Spawn the ego vehicle (Tesla Model 3) at the first spawn point
# --------------------------------------------------------------
blueprints = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
vehicle_bp = blueprints.find("vehicle.tesla.model3")
vehicle = world.spawn_actor(vehicle_bp, spawn_points[0])

# --------------------------------------------------------------
# Attach an RGB camera sensor to the vehicle
# --------------------------------------------------------------
rgb_bp = blueprints.find("sensor.camera.rgb")
rgb_bp.set_attribute("image_size_x", "1280")   # width in pixels
rgb_bp.set_attribute("image_size_y", "720")    # height in pixels
rgb_bp.set_attribute("fov", "90")              # horizontal field of view
camera_transform = carla.Transform(
    carla.Location(x=0.3, z=1.6),              # position relative to car
    carla.Rotation(pitch=-6)                   # slightly facing downward
)
rgb_cam = world.spawn_actor(rgb_bp, camera_transform, attach_to=vehicle)

# --------------------------------------------------------------
# Cleanup routine (automatically called on exit)
# Ensures all actors are properly destroyed
# --------------------------------------------------------------
def cleanup():
    try:
        rgb_cam.stop()
        vehicle.set_autopilot(False)
        time.sleep(0.1)
        rgb_cam.destroy()
        vehicle.destroy()
        settings.synchronous_mode = False
        world.apply_settings(settings)
        cv2.destroyAllWindows()
    except:
        pass
atexit.register(cleanup)

# --------------------------------------------------------------
# Listen to camera frames asynchronously
# The last received frame is stored in the "frame" dict
# --------------------------------------------------------------
frame = {"img": None}
rgb_cam.listen(lambda img: frame.update(img=img))

# Enable autopilot so the vehicle drives automatically
vehicle.set_autopilot(True)

print("Running... (Press ESC to stop)")
# --------------------------------------------------------------
# Main simulation loop
# --------------------------------------------------------------
while True:
    world.tick()                    # advance the simulation one frame
    if frame["img"] is None:        # skip until the first image arrives
        continue

    # Convert CARLA image to a normal RGB NumPy array
    rgb = carla_image_to_rgb(frame["img"])

    # ----------------------------------------------------------
    # Run YOLO detection on the current RGB frame
    # Only draw bounding boxes (no class labels yet)
    # ----------------------------------------------------------
    res = model.predict(rgb, verbose=False, conf=0.25)[0]

    if res.boxes is not None and len(res.boxes) > 0:
        # Iterate over all detected boxes
        for (x1, y1, x2, y2) in res.boxes.xyxy.cpu().numpy():
            # Draw green rectangle around each detected object
            p1 = (int(x1), int(y1))
            p2 = (int(x2), int(y2))
            cv2.rectangle(rgb, p1, p2, (0, 255, 0), 2)

    # Show the annotated image in a live OpenCV window
    cv2.imshow("Detection only", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR))

    # Exit loop when ESC key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

<h3>Training the YOLOv11 </h3>